In [1]:
import openferro as of
from openferro.interaction import *
from openferro.simulation import *

In [2]:
latt = of.BravaisLattice3D(3, 3, 3)
pto = of.System(latt, pbc=True)
dipole_field = pto.add_field(name="dipole", ftype="scalar", value=0.75)

double_well = self_energy_R1_onsite
pto.add_self_interaction('double_well', field_name="dipole", energy_engine=double_well, parameters={'k2': -1, 'alpha': 1, 'offset': 0}, enable_jit=True)

ising_couple = self_energy_R1_neighbor_pbc
pto.add_self_interaction('ising_couple', field_name="dipole", energy_engine=ising_couple, parameters={'J1': -0.1, 'J2': -0.1, 'J3': -0.1, 'offset': 0}, enable_jit=True)

print('mass:', pto.get_field_by_name('dipole').get_mass().flatten())
pto.update_force()
print('max force before optimization',pto.get_field_by_name("dipole").get_force().max())
print('config before optimization',pto.get_field_by_name("dipole").get_values().flatten())
minimizer = MDMinimize(pto, max_iter=100, tol=1e-5, dt=0.1)
minimizer.minimize()
print('max force after optimization',pto.get_field_by_name("dipole").get_force().max())
print('config after optimization',pto.get_field_by_name("dipole").get_values().flatten())

mass: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.]
max force before optimization 0.26250002
config before optimization [0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75
 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75]
max force after optimization 5.7816505e-06
config after optimization [0.80622524 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524
 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524
 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524
 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524
 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524
 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524
 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524
 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524
 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524 0.80622524
 0.80622524 0.80622524 0.8062252

In [13]:

# simulation = SimulationNVE(pto, dt=0.01, temperature=1)
simulation = SimulationNVTLangevin(pto, dt=0.001, temperature=100, gamma=0.1)
simulation.init_velocity(mode='gaussian')
# print('init position:', pto.get_field_by_name('dipole').get_values().flatten())
# print('init velocity:', pto.get_field_by_name('dipole').get_velocity().flatten())


In [14]:
for idx in range(10):
    %timeit simulation.step(10)
    # print('velocity:', pto.get_field_by_name('dipole').get_velocity().flatten())
    # print('field:', pto.get_field_by_name('dipole').get_values().flatten())
    pot_E = pto.calc_potential_energy()
    kin_E = pto.calc_kinetic_energy()
    total_E = pot_E + kin_E
    average_field = pto.get_field_by_name('dipole').get_values().mean()
    print('=============================================')
    print('pot energy:', pot_E)
    print('kine energy:', kin_E)
    print('total energy:', total_E)
    print('temperature:', pto.calc_temp_by_name('dipole'))
    print('average field:', average_field)
    # print('field:', pto.get_field_by_name('dipole').get_values().flatten())

258 ms ± 6.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
pot energy: -33.76358
kine energy: 0.36095086
total energy: -33.40263
temperature: 103.42375
average field: 0.8118404
258 ms ± 2.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
pot energy: -33.784454
kine energy: 0.37682423
total energy: -33.40763
temperature: 107.97197
average field: 0.81058174
256 ms ± 1.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
pot energy: -33.81161
kine energy: 0.4050375
total energy: -33.406574
temperature: 116.055954
average field: 0.8091864
257 ms ± 2.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
pot energy: -33.841824
kine energy: 0.43941316
total energy: -33.40241
temperature: 125.90567
average field: 0.8077596
258 ms ± 3.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
pot energy: -33.867897
kine energy: 0.47626245
total energy: -33.391636
temperature: 136.46416
average field: 0.8062892
260 ms ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop eac